In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
import wordcloud
import string
import validators
import re
import nltk
import langdetect
from collections import Counter
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 100

In [ ]:
mac_mic = glob.glob("./Data/*.csv")
influencers = pd.DataFrame()
for i in mac_mic:
    movement = i.split("/")[2].split(".")[0].split("_")[2]
    df = pd.read_csv(i)
    print(df.shape)
    if movement == "sb":
        df["movement_name"] = "StopTheBan"
    elif movement == "gw":
        df["movement_name"] = "GrabYourWallet"
    elif movement == "wm":
        df["movement_name"] = "WhyIMarch"
    else:
        df["movement_name"] = "McStrike"
    influencers = pd.concat([influencers,df])

In [ ]:
print("Total macro-micro influencers :- ",influencers.shape,influencers.columns)

In [ ]:
influencers.to_csv("./influencers.csv")

In [ ]:
dataM1 = glob.glob("./Data/20170122 WhyIMarch/*.csv")
dataM2 = glob.glob("./Data/20170204 GrabYourWallet/*.csv")
dataM3 = glob.glob("./Data/20190521 StoptheBan/*.csv")
dataM4 = glob.glob("./Data/McStrike/*.csv")

M1 = pd.DataFrame()
for d in dataM1:
    M1 = pd.concat([M1,pd.read_csv(d)])

M2 = pd.DataFrame()
for d in dataM2:
    M2 = pd.concat([M2,pd.read_csv(d)])
    
M3 = pd.DataFrame()
for d in dataM3:
    M3 = pd.concat([M3,pd.read_csv(d)])
    
M4 = pd.DataFrame()
for d in dataM4:
    M4 = pd.concat([M4,pd.read_csv(d)])    
    
M1.columns = [x.replace("[M] ","").replace(": ","") for x in M1.columns] 
M2.columns = [x.replace("[M] ","").replace(": ","") for x in M2.columns] 
M3.columns = [x.replace("[M] ","").replace(": ","") for x in M3.columns] 
M4.columns = [x.replace("[M] ","").replace(": ","") for x in M4.columns] 

M1["followers_count"] = M1["followers_count"].fillna(0)
M2["followers_count"] = M2["followers_count"].fillna(0)
M3["followers_count"] = M3["followers_count"].fillna(0)

M1["favourites_count"] = M1["favourites_count"].fillna(0)
M2["favourites_count"] = M2["favourites_count"].fillna(0)
M3["favourites_count"] = M3["favourites_count"].fillna(0)

M1["friends_count"] = M1["friends_count"].fillna(0)
M2["friends_count"] = M2["friends_count"].fillna(0)
M3["friends_count"] = M3["friends_count"].fillna(0)

M1["username"] = M1["username"].fillna("NA")
M2["username"] = M2["username"].fillna("NA")
M3["username"] = M3["username"].fillna("NA")

M1["hashtag"] = M1["hashtag"].fillna("NA")
M2["hashtag"] = M2["hashtag"].fillna("NA")                                   
M3["hashtag"] = M3["hashtag"].fillna("NA")

M1["movement_name"] = "WhyIMarch"
M2["movement_name"] = "GrabYourWallet"
M3["movement_name"] = "StopTheBan"
M4["movement_name"] = "McStrike"

collective = pd.concat([M1,M2,M3,M4])
collective["created_at"] =  pd.to_datetime(collective["created_at"], format='%Y-%m-%d %H:%M:%S')
collective = collective.drop(["Annotations","ReferenceText"],axis=1)
#collective = collective[collective["Text"].str.contains("^RT")]

print(collective.columns, collective.shape)
collective = shuffle(collective)

collective["is_retweet"] = 0
collective.loc[collective["Text"].str.contains("^RT"),"is_retweet"] = 1
print("Retweet distribution :- \n",collective["is_retweet"].value_counts())

In [ ]:
collective.to_csv("./collective.csv")

In [2]:
collective = pd.read_csv("./collective.csv")
influencers = pd.read_csv("./influencers.csv")
influencers =influencers.drop("Unnamed: 0",axis=1)
influencers

,name,community,macro_micro,movement_name
0,sarahfelts,5,macro,StopTheBan
1,PPMO_Advocates,5,macro,StopTheBan
2,jeffssore,5,macro,StopTheBan
3,wrap02,5,macro,StopTheBan
4,sharingjoy91,5,macro,StopTheBan
...,...,...,...,...
52399,vallyweeknd,108,micro,McStrike
52400,TheKhlimax,108,micro,McStrike
52401,lanre_okunlola,108,micro,McStrike
52402,SalesRubyNG,131,micro,McStrike


In [3]:
categories = {"is_media":["media","politics","journalism","journalist","news","reporter"],  
             "is_education":["educationist","teacher","professor","tutor","pedagogue","scientist","instructor", 
                               "coach","trainer","lecturer","academic","counselor"], 
             "is_politics":["politician","politics"]}
             #"is_activist":["activist"], # "is_community":["organize","community","volunteer"]} #

In [4]:
def add_category_columns(row):
    desc = row["user_description"]
    if desc is not np.nan:
        for c in categories:
            row[c] = int(0)
            for val in c:
                if val in desc:
                    row[c] = int(1)
    return row

In [5]:
all_user_details = {}
for i,j in collective[["from_user","user_description"]].values:
    if i not in all_user_details:
        all_user_details[i] = j
all_user_details = pd.DataFrame(all_user_details.items(), columns=["from_user", "user_description"])
print(all_user_details.shape)
all_user_details.head()

(356324, 2)


,from_user,user_description
0,TwiitterStinks,Joined in 2011. Shadow banned. Comment ghosted...
1,issalitesnack,Thee Paul Martiñ 😎
2,sba1211,NaN
3,menokingirl,"artsy, sculpture/glass, funky/shoe-gazing/ecle..."
4,SallySuePrescot,Peace Love Kumbaya MAGA KAG


In [6]:
movements = collective.groupby("from_user")["movement_name"].apply(set)

In [7]:
all_user_details = pd.merge(all_user_details,movements,on="from_user")

In [8]:
all_user_details.head()

,from_user,user_description,movement_name
0,TwiitterStinks,Joined in 2011. Shadow banned. Comment ghosted...,{McStrike}
1,issalitesnack,Thee Paul Martiñ 😎,{McStrike}
2,sba1211,NaN,"{McStrike, WhyIMarch, GrabYourWallet, StopTheBan}"
3,menokingirl,"artsy, sculpture/glass, funky/shoe-gazing/ecle...",{WhyIMarch}
4,SallySuePrescot,Peace Love Kumbaya MAGA KAG,{McStrike}


In [9]:
all_user_details.isnull().sum()

from_user               0
user_description    56923
movement_name           0
dtype: int64

In [10]:
all_user_details.shape

(356324, 3)

In [11]:
merged = pd.merge(influencers,all_user_details,right_on="from_user",left_on="name",how="left")

In [13]:
print("Number of influencers with description :- ",len(set(influencers["name"])&(set(all_user_details["from_user"]))))

Number of influencers with description :-  49143


In [14]:
influencers.shape

(52404, 4)

In [15]:
len(influencers["name"].unique())

51018

In [18]:
merged.rename(columns = {'movement_name_x':'influenced_movement','movement_name_y':'movements_participated'}, inplace = True)

In [20]:
merged.isnull().sum(),merged.shape

(name                         0
 community                    0
 macro_micro                  0
 influenced_movement          0
 from_user                 1887
 user_description          9273
 movements_participated    1887
 dtype: int64, (52404, 7))

In [21]:
merged = pd.merge(merged,collective.loc[collective["from_user"].isin(merged["from_user"])].sort_values("created_at").groupby("from_user").last().reset_index()[["from_user","created_at","followers_count"]],how="left")

In [22]:
merged = merged.apply(add_category_columns,axis=1)

KeyboardInterrupt: 

In [ ]:
merged.head()

In [ ]:
mention_screen_names = []
for i in list(collective["user_mention_screen_name"]):
    try:
        i = i.split(";")
        mention_screen_names.append([x.strip() for x in i])
    except Exception as e:
        pass
# s = collective[collective["Text"].str.contains("^RT")]["Text"].iloc[10]
# s[s.find("@"):s.find(":")]

mentions = {}
for i in mention_screen_names:
    for j in i:
        if j not in mentions.keys():
            mentions[j] = 1
        else:
            mentions[j] += 1

In [ ]:
def get_mention_counts(row):
    user = row["name"]
    if user in mentions:
        row["mention_count"] = mentions[user]
    return row

In [ ]:
merged = merged.apply(get_mention_counts,axis=1)

In [ ]:
merged.columns

In [ ]:
merged.isnull().sum()

In [ ]:
print(merged["is_education"].value_counts(),"\n",merged["is_media"].value_counts(),"\n",merged["is_politics"].value_counts())

In [ ]:
merged = shuffle(merged)

In [ ]:
def detect_lang(row):
    #text = row["Text"]
    text = row["user_description"]
    row["bio_language"] = "NA"
    if text is not np.nan: 
        #is_url = validators.url(text)
        try:
            #if len(text) == 0 or is_url == False:
                row["bio_language"] = langdetect.detect(text)
        except Exception as e:
            pass
    return row

In [ ]:
tweet_counts = collective.groupby("from_user")["Text"].count().reset_index()

In [ ]:
tweet_counts.columns = ["from_user","tweets_count"]

In [ ]:
merged = pd.merge(merged,tweet_counts[tweet_counts["from_user"].isin(merged["from_user"])],how="left",on="from_user")

In [ ]:
merged.head()

In [ ]:
collective[collective["is_retweet"]==1]["Text"].iloc[8].split(" ")[1].replace("@","").replace(":","")

In [ ]:
retweet_user_list = []
for t in collective[collective["is_retweet"]==1]["Text"]:
    retweet_user_list.append(t.split(" ")[1].replace("@","").replace(":",""))
print(len(retweet_user_list))
retweet_users = dict(Counter(retweet_user_list))

In [ ]:
def get_retweet_count(row):
    row["retweet_count"] = 0
    if row["from_user"] in retweet_users:
        row["retweet_count"] = retweet_users[row["from_user"]]
    return row

In [ ]:
len(list(collective["Text"].unique())),len(list(collective["Text"])) 

In [ ]:
tweets = list(collective["Text"].unique())

In [ ]:
def cleanText(text):
    text = re.sub('@[A-Za-z0–9]+', '', text)          # Removing @mentions
    text = re.sub('#', '', text)                      # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text)                # Removing RT
    text = re.sub('https?:\/\/\S+', '', text)         # Removing hyperlink
    return text

In [ ]:
# for_tok = collective[collective["from_user"].isin(merged["from_user"])][["from_user","movement_name","Text"]]

In [ ]:
sample = collective.head(20000).reset_index(drop=True)
sample = sample.apply(add_category_columns,axis=1)
sample["length_of_text"] = sample["Text"].str.len()
sample = sample.apply(detect_lang,axis=1)
sample = sample[sample["language"] == "en"]
print("Sample shape after removing non-english tweets :- ",sample.shape)
print("Number of unique users :- ",len(sample["from_user"].unique()))

user_details = {}
for i, j in sample[["from_user","user_description"]].values:
    if i not in user_details:
        user_details[i] = j
user_details = pd.DataFrame(user_details.items(), columns=['from_user', 'user_description'])
print(user_details.shape)
user_details.head()
user_details = pd.merge(user_details,sample.loc[sample.groupby('from_user').created_at.idxmax()][["from_user","created_at","movement_name","is_media","is_education","is_politics","followers_count"]])

In [ ]:
# create a vocabulary of words, 
# ignore words that appear in 85% of documents, 
# eliminate stop words
cv = CountVectorizer(max_df = 0.85,stop_words = stopwords)
word_count_vector = cv.fit_transform(descriptions)
print("Number of documents and vocabulary length :- ",word_count_vector.shape)

In [ ]:
all_descriptions = {}
for i, j in collective[["from_user","user_description"]].values:
    if i not in all_descriptions:
        all_descriptions[i] = j
descriptions = []
for i in list(all_descriptions.values()):
    if i is not np.nan:
        descriptions.append(i)
        
stopwords = list(wordcloud.STOPWORDS)
stopwords.extend(["https","co","everything"])
words = []
for idx,i in enumerate(list(all_descriptions.values())):
    if i is not np.nan:
        i = i.replace(" https ","").replace(" co ","")
        w = i.split(" ")
        for j in w:
            j = j.lower()
            if j not in stopwords \
            and not j.startswith("@") \
            and j != "rt" \
            and j not in string.punctuation \
            and j != "https" and len(j) > 2:
                words.append(j)
word_string = ' '.join([x for x in words])
word_string = word_string.replace("https","")

wc = wordcloud.WordCloud(
                      stopwords=stopwords,
                      background_color='black',
                      width=3000,
                      height=2000,
                      max_words=200
                     ).generate(word_string)

plt.imshow(wc)
plt.axis('off')
plt.savefig('./my_twitter_wordcloud_1.png', dpi=300)
plt.show()